## What was discovered?
* is_tourney (remark when game was in regular or tourney phase) variable doesnt matter according to feature importance and also we will predict only tourney phase games.
* to addres the problem of data leakage I removed scores from games (before game we dont know which teams shoot how many points) and also for every team i retrieve seed values. But if it is regular season i only have information about previous ones. So for regular season I retrieve seed from previous season and for tourney phase i retrieve information from current season.
* team coaches can have also some important informations (some coaches are better than other). But during the season team coache can be changed. So apart classic merge I have to check also if in particular row daynum is smaller than last day and bigger than first day of coach term.
* DayNum feature can carry some information. But in this contest we do not have information about matches (we have to predict every possible matchup). Feature importance indicate it is important feature.
* Ranking provide a lot of information. In data we have a massive ranking, which contains a team's ranking from a lot of rankings. There is above 100 rankings. So i will caluclate median for certain game for particular team. Also we dont have rankings for every daynum, so i merged data on teams ID, Season and DayNum, then groupby this columns and i take the last one element. Therefor I will have always the last one. To prevent data leakage I removed all rows contain ranking after the daynum of game. So I excluded information which I won't have during predicitons. As we can see the mutual information and random forest indicate it is a important feature.
* Next features are very important, because number of wins and average number of scored points can be really helpful. As humans try to bet on games we often checks on which position team is, how many wins have or how many scored points. So it can convey a lot of information.

## Which features was added?
* seed value for every team
* difference between seed of one team and another
* team coach information
* duration of teams presence in Division I
* ranking's median
* average amount of scored points (in particular season - if it is a 15 game for a team, mean will be calculate for previous 14)
* number of wins in particular season (if it is a 15 game, sum will be calucalted from 14 previous games)

## Load libraries

In [1]:
import fireducks.pandas as pd
import numpy as np
from sklearn.metrics import brier_score_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from custom_utils import transform_data, enrich_data, feature_importance
from IPython.display import display
from sklearn.decomposition import PCA

## Load and prepare data

In [2]:
wregularseason = pd.read_csv("data/WRegularSeasonCompactResults.csv")
wtourneyseason = pd.read_csv("data/WNCAATourneyCompactResults.csv")

mregularseason = pd.read_csv("data/MRegularSeasonCompactResults.csv")
mtourneyseason = pd.read_csv("data/MNCAATourneyCompactResults.csv")

mseason = pd.read_csv("data/MSeasons.csv")
wseason = pd.read_csv("data/WSeasons.csv")

wdata = pd.concat([wregularseason, wtourneyseason], axis=0)
wdata = wdata.loc[wdata.Season > 2015].reset_index(drop=True)

mdata = pd.concat([mregularseason, mtourneyseason], axis=0).reset_index(drop=True)
mdata = mdata.loc[mdata.Season > 2015].reset_index(drop=True)

wprep = transform_data(wdata)
mprep = transform_data(mdata)

In [3]:
mprep_enh = enrich_data(mprep, "M")
wprep_enh = enrich_data(wprep, "W")
display(wprep_enh.head())
mprep_enh.head()

/home/kacper/Projects/Forecasting-NCAA-BB/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/kacper/Projects/Forecasting-NCAA-BB/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,Season,DayNum,WTeamID,LTeamID,Result,LScore,WScore,Lwin,Wwin,is_tournament,SeedW,SeedL,SeedDiff
0,2024,110,3328,3329,1,70.538462,77.269231,13,19,0,5,8,-3
1,2024,110,3452,3124,0,76.115385,76.884615,6,22,0,10,7,3
2,2024,110,3437,3166,0,73.280000,65.692308,4,17,0,4,6,-2
3,2024,110,3329,3328,0,77.269231,70.538462,7,13,0,8,5,3
4,2024,101,3397,3376,0,0.000000,0.000000,0,0,0,4,1,3


,Season,DayNum,WTeamID,LTeamID,Result,LScore,WScore,Lwin,Wwin,is_tournament,SeedW,SeedL,WCoachName,LCoachName,Duration,RankW,RankL,SeedDiff
0,2016,16,1103,1116,1,0.0,0.0,0,0,0,16,5,368,471,32.0,77.0,45.0,11
1,2016,16,1113,1244,1,66.5,73.0,2,1,0,16,16,65,6,32.0,86.0,334.0,0
2,2016,16,1116,1103,0,0.0,0.0,0,0,0,5,16,471,368,32.0,45.0,77.0,-11
3,2016,16,1117,1366,0,0.0,0.0,0,0,0,16,16,331,257,32.0,268.0,334.0,0
4,2016,16,1119,1192,1,54.0,75.0,1,1,0,16,16,694,245,32.0,193.0,324.0,0


## Test models on different features

### Divide dataset on X and y, define time series split function

In [4]:
N_COMPONENTS = 5
pca = PCA(n_components=N_COMPONENTS)

feature_labels = [f"feature_{i}" for i in range(1, N_COMPONENTS + 1)]

X_featm = mprep_enh.drop(["Result", "DayNum"], axis=1)
ym = mprep_enh.Result

X_featw = wprep_enh.drop(["Result", "DayNum"], axis=1)
yw = wprep_enh.Result

tscv = TimeSeriesSplit(n_splits=10)

X_trainm, X_testm, y_trainm, y_testm = train_test_split(X_featm, ym, test_size=0.1)
X_trainw, X_testw, y_trainw, y_testw = train_test_split(X_featw, yw, test_size=0.1)

X_trainm_pca = pd.DataFrame(pca.fit_transform(X_trainm), columns=feature_labels)
X_testm_pca = pd.DataFrame(pca.transform(X_testm), columns=feature_labels)

X_trainw_pca = pd.DataFrame(pca.fit_transform(X_trainw), columns=feature_labels)
X_testw_pca = pd.DataFrame(pca.transform(X_testw), columns=feature_labels)

### Without PCA

In [8]:
brier_score_cvm = []
brier_score_cvw = []

for train_idx, test_idx in tscv.split(X_featm):
    X_train, y_train = X_featm.iloc[train_idx], ym.iloc[train_idx]
    X_test, y_test = X_featm.iloc[test_idx], ym.iloc[test_idx]

    rf_clfm = RandomForestClassifier()
    rf_clfm.fit(X_train, y_train)

    pred_probsm = rf_clfm.predict_proba(X_test)[:, 1]
    brier_score_cvm.append(brier_score_loss(y_test, pred_probsm).item())

for train_idx, test_idx in tscv.split(X_featw):
    X_train, y_train = X_featw.iloc[train_idx], yw.iloc[train_idx]
    X_test, y_test = X_featw.iloc[test_idx], yw.iloc[test_idx]

    rf_clfw = RandomForestClassifier()
    _ = rf_clfw.fit(X_train, y_train)

    pred_probsw = rf_clfw.predict_proba(X_test)[:, 1]
    brier_score_cvw.append(brier_score_loss(y_test, pred_probsw).item())

print(f"Brier score for mens data (CV on 10 split): {np.mean(brier_score_cvm).item()}")
print(f"Brier score for womens data (CV on 10 split): {np.mean(brier_score_cvw).item()}")

Brier score for mens data (CV on 10 split): 0.205564961513207
Brier score for womens data (CV on 10 split): 0.20885259758589064


In [9]:
print("For men's games")
display(feature_importance(rf_clfm, mprep_enh.drop("DayNum", axis=1)))

print("For women's games")
display(feature_importance(rf_clfw, wprep_enh.drop("DayNum", axis=1)))

For men's games


,Feature,Importance_clf,Importance_mi
0,Season,0.031961,0.002574
1,WTeamID,0.065683,0.027388
2,LTeamID,0.067351,0.027709
3,LScore,0.100823,0.019789
4,WScore,0.104112,0.019591
5,Lwin,0.063323,0.014454
6,Wwin,0.063156,0.014171
7,is_tournament,0.000631,0.000000
8,SeedW,0.012964,0.019676
9,SeedL,0.013530,0.018404


For women's games


,Feature,Importance_clf,Importance_mi
0,Season,0.067303,0.001737
1,WTeamID,0.164019,0.043360
2,LTeamID,0.162406,0.045103
3,LScore,0.170269,0.032832
4,WScore,0.174866,0.030507
5,Lwin,0.079690,0.017091
6,Wwin,0.077096,0.017705
7,is_tournament,0.000000,0.001253
8,SeedW,0.023551,0.024679
9,SeedL,0.024152,0.027384


### With PCA

In [ ]:
brier_score_cvm = []
brier_score_cvw = []

N_COMPONENTS = 5
feature_labels = [f"feature_{i}" for i in range(1, N_COMPONENTS + 1)]

for train_idx, test_idx in tscv.split(X_featm):
    pca = PCA(n_components=N_COMPONENTS)
    X_train, y_train = X_featm.iloc[train_idx], ym.iloc[train_idx]
    X_test, y_test = X_featm.iloc[test_idx], ym.iloc[test_idx]

    X_trainpca = pd.DataFrame(pca.fit_transform(X_train), columns=feature_labels)
    X_testpca = pd.DataFrame(pca.transform(X_test), columns=feature_labels)

    rf_clfm = RandomForestClassifier()
    rf_clfm.fit(X_trainpca, y_train)

    pred_probsm = rf_clfm.predict_proba(X_testpca)[:, 1]
    brier_score_cvm.append(brier_score_loss(y_test, pred_probsm).item())

for train_idx, test_idx in tscv.split(X_featw):
    pca = PCA(n_components=N_COMPONENTS)
    X_train, y_train = X_featw.iloc[train_idx], yw.iloc[train_idx]
    X_test, y_test = X_featw.iloc[test_idx], yw.iloc[test_idx]

    X_trainpca = pd.DataFrame(pca.fit_transform(X_train), columns=feature_labels)
    X_testpca = pd.DataFrame(pca.transform(X_test), columns=feature_labels)

    rf_clfw = RandomForestClassifier()
    _ = rf_clfw.fit(X_trainpca, y_train)

    pred_probsw = rf_clfw.predict_proba(X_testpca)[:, 1]
    brier_score_cvw.append(brier_score_loss(y_test, pred_probsw).item())

print(f"Brier score for mens data (CV on 10 split): {np.mean(brier_score_cvm).item()}")
print(f"Brier score for womens data (CV on 10 split): {np.mean(brier_score_cvw).item()}")

AttributeError: 'list' object has no attribute 'values'

In [ ]:
wdatatrain = pd.concat([X_trainw_pca, y_trainw.reset_index(drop=True)], axis=1)
mdatatrain = pd.concat([X_trainm_pca, y_trainm.reset_index(drop=True)], axis=1)

wdatatest = pd.concat([X_testw_pca, y_testw.reset_index(drop=True)], axis=1)
mdatatest = pd.concat([X_testm_pca, y_testm.reset_index(drop=True)], axis=1)

wdata = pd.concat([wdatatrain, wdatatest], axis=0).reset_index(drop=True)
mdata = pd.concat([mdatatrain, mdatatest], axis=0).reset_index(drop=True)

print("For men's games")
display(feature_importance(rf_clfm, mdata))

print("For women's games")
display(feature_importance(rf_clfw, wdata))

For men's games


,Feature,Importance_clf,Importance_mi
0,feature_1,0.201838,0.023034
1,feature_2,0.199936,0.023934
2,feature_3,0.196571,0.002377
3,feature_4,0.203715,0.009658
4,feature_5,0.197940,0.003535


For women's games


,Feature,Importance_clf,Importance_mi
0,feature_1,0.160620,0.061355
1,feature_2,0.160613,0.053347
2,feature_3,0.169058,0.006719
3,feature_4,0.329009,0.098678
4,feature_5,0.180701,0.024859
